In [88]:
import pandas as pd
import csv
import pymongo
import json
%load_ext blackcellmagic
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, desc, inspect, cast, Float,Integer
from db_config import engine, DATABASE

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [89]:
# engine.execute(f"USE {DATABASE}")
global_sql = "SELECT * FROM global_covid_data"

In [6]:
df = pd.read_sql_query(global_sql, engine)

In [7]:
global_covid_data = pd.DataFrame(
    df.groupby(["Country_Region", "Date"]).agg(
        {"Confirmed_Cases": "sum", "Deaths": "sum", "Recovered": "sum"}
    )
)
global_covid_data

Confirmed_Cases  Deaths  Recovered
Country_Region Date                                       
Afghanistan    1/22/20                0     0.0        0.0
               1/23/20                0     0.0        0.0
               1/24/20                0     0.0        0.0
               1/25/20                0     0.0        0.0
               1/26/20                0     0.0        0.0
...                                 ...     ...        ...
Zimbabwe       4/5/20                 9     1.0        0.0
               4/6/20                10     1.0        0.0
               4/7/20                11     2.0        0.0
               4/8/20                11     3.0        0.0
               4/9/20                11     3.0        0.0

[14536 rows x 3 columns]

In [90]:
inspector = inspect(db.engine)
table_names = inspector.get_table_names()

# schema = inspector.get_columns("icecreamstore")
# print(schema)
table_names

['global_covid_data', 'us_covid_data', 'us_lookup']

In [133]:
Base = automap_base()
Base.prepare(engine, reflect=True)
global_data = Base.classes.global_covid_data
us_data = Base.classes.us_covid_data
us_lookup = Base.classes.us_lookup
session = Session(engine)

In [134]:
schema = inspector.get_columns("global_covid_data")
# schema

In [8]:
countries = session.query(global_data.Country_Region).distinct().order_by(global_data.Country_Region).all()

In [9]:
global_covid_results = (
    session.query(
        global_data.Country_Region,
        global_data.Date,
        func.sum(global_data.Confirmed_Cases),
        func.sum(global_data.Deaths),
        func.sum(global_data.Recovered),
    )
    .group_by(global_data.Country_Region, global_data.Date)
    .order_by(global_data.Country_Region)
).all()

In [10]:
def get_country_covid_data(country):
    data = {}
    for rec in global_covid_results:
        if rec[0] == country:
            data[rec[1]] = {"Confirmed_Cases" : int(rec[2]), "Deaths" : int(rec[3]), "Recovered" : int(rec[4])}
    return data

In [11]:
global_dict  = [{  "Country": country[0],
     "Data" : get_country_covid_data(country[0])
  }for country in countries]
global_dict

[{'Country': 'Afghanistan',
  'Data': {'1/22/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '1/23/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '1/24/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '1/25/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '1/26/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '1/27/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '1/28/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '1/29/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '1/30/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '1/31/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '2/1/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '2/10/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '2/11/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '2/12/20': {'Confirmed_Cases': 0, 'Deaths': 0, 'Recovered': 0},
   '2/13/20': {'Confirmed_C

In [121]:
country = "US"
data_point = "Deaths"


In [137]:

country_results = (
    session.query(
        global_data.ID,
        global_data.Country_Region,
        global_data.Date,
        func.sum(global_data.Confirmed_Cases),
        func.sum(global_data.Deaths),
        func.sum(global_data.Recovered)
    ).filter(global_data.Country_Region == country)
     .group_by(global_data.ID, global_data.Country_Region, global_data.Date)
    .order_by(global_data.Country_Region)
).all()

In [138]:
results = [{"Date": rec[1], data_point: int(rec[col_idx])} for rec in country_results]

ValueError: invalid literal for int() with base 10: '1/22/20'

In [145]:
qry =(
    session.query(
        global_data.ID,
        global_data.Country_Region,
        global_data.Date,
        func.sum(global_data.Confirmed_Cases),
        func.sum(global_data.Deaths),
        func.sum(global_data.Recovered)
    ).filter(global_data.Country_Region == country)
     .group_by(global_data.ID, global_data.Country_Region, global_data.Date)
    .order_by(global_data.ID)
).statement

In [146]:
df = pd.read_sql_query(qry, engine)
df

,ID,Country_Region,Date,sum_1,sum_2,sum_3
0,225,US,1/22/20,1.0,0.0,0.0
1,488,US,1/23/20,1.0,0.0,0.0
2,751,US,1/24/20,2.0,0.0,0.0
3,1014,US,1/25/20,2.0,0.0,0.0
4,1277,US,1/26/20,5.0,0.0,0.0
...,...,...,...,...,...,...
74,19687,US,4/5/20,337072.0,9619.0,17448.0
75,19950,US,4/6/20,366667.0,10783.0,19581.0
76,20213,US,4/7/20,396223.0,12722.0,21763.0
77,20476,US,4/8/20,429052.0,14695.0,23559.0


In [131]:
df_col = ""
if data_point == "Deaths":
    dc_col = df.sum_2.values.tolist()
elif data_point == "Recovered":
    df_col = df.sum_3.values.tolist()
else:
    df_col = df.sum_1.values.tolist()
data = {
    "Date": df.Date.values.tolist(),
    "Confirmed_Cases": df.sum_1.values.tolist(),
    "Deaths": df.sum_2.values.tolist(),
    "Recovered": df.sum_3.values.tolist(),
}

In [132]:
data

{'Date': ['1/22/20',
  '1/23/20',
  '1/24/20',
  '1/25/20',
  '1/26/20',
  '1/27/20',
  '1/28/20',
  '1/29/20',
  '1/30/20',
  '1/31/20',
  '2/1/20',
  '2/2/20',
  '2/3/20',
  '2/4/20',
  '2/5/20',
  '2/6/20',
  '2/7/20',
  '2/8/20',
  '2/9/20',
  '2/10/20',
  '2/11/20',
  '2/12/20',
  '2/13/20',
  '2/14/20',
  '2/15/20',
  '2/16/20',
  '2/17/20',
  '2/18/20',
  '2/19/20',
  '2/20/20',
  '2/21/20',
  '2/22/20',
  '2/23/20',
  '2/24/20',
  '2/25/20',
  '2/26/20',
  '2/27/20',
  '2/28/20',
  '2/29/20',
  '3/1/20',
  '3/2/20',
  '3/3/20',
  '3/4/20',
  '3/5/20',
  '3/6/20',
  '3/7/20',
  '3/8/20',
  '3/9/20',
  '3/10/20',
  '3/11/20',
  '3/12/20',
  '3/13/20',
  '3/14/20',
  '3/15/20',
  '3/16/20',
  '3/17/20',
  '3/18/20',
  '3/19/20',
  '3/20/20',
  '3/21/20',
  '3/22/20',
  '3/23/20',
  '3/24/20',
  '3/25/20',
  '3/26/20',
  '3/27/20',
  '3/28/20',
  '3/29/20',
  '3/30/20',
  '3/31/20',
  '4/1/20',
  '4/2/20',
  '4/3/20',
  '4/4/20',
  '4/5/20',
  '4/6/20',
  '4/7/20',
  '4/8/20',
  '4